In [1]:
import os
from os.path import join
import csv
import cv2
import numpy as np

from keras.layers import Input, Conv2D, Dropout, MaxPool2D, GlobalAveragePooling2D, GlobalMaxPooling2D, LeakyReLU
from keras.layers import concatenate, Dense
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

from paths import PATH_DATA, PATH_PROJECT
from iterators import roundrobin, repeat_infinitely

Using TensorFlow backend.


In [2]:
class DarknetBlock:
    def __init__(self, filters1, filters3, strides=(1, 1)):
        self._filters1 = filters1
        self._filters3 = filters3
        self._strides = strides
        
    def __call__(self, input_layer, *args, **kwargs):
        x = input_layer
        for filters, kernel, strides in zip([self._filters1, self._filters3],
                                            [(1, 1), (3, 3)],
                                            [(1, 1), self._strides]):
            x = Conv2D(filters, kernel, padding='same', strides=strides)(x)
            x = LeakyReLU()(x)
        return x


def build_model(class_count, is_train=False):
    input_tensor = Input(shape=(1, None, None))
    x = Conv2D(10, (3, 3), strides=(2, 2), padding='same')(input_tensor)
    filters = [20, 30, 40]
    for idx, filter_size in enumerate(filters):
        x = DarknetBlock(filter_size, filter_size * 2)(x)
        x = DarknetBlock(filter_size, filter_size * 2)(x)
        if idx < len(filters) - 1:
            x = MaxPool2D(pool_size=(2, 2), padding='same')(x)
            if is_train:
                x = Dropout(0.1)(x)
    
    x = Conv2D(filters[-1], (1, 1), padding='same')(x)
    max_output = GlobalMaxPooling2D()(x)
    average_output = GlobalAveragePooling2D()(x)
    x = concatenate(inputs=[max_output, average_output], axis=1)
    x = Dense(class_count, activation='softmax')(x)
    return Model(inputs=[input_tensor], outputs=[x])

In [3]:
# cannot make bigger batches because images are of different size
BATCH_SIZE = 1

SQUARE_COUNT = 3000
TRAIN_SQUARE_SIZE = SQUARE_COUNT * 80 // 100
VAL_SQUARE_SIZE = SQUARE_COUNT - TRAIN_SQUARE_SIZE

RECT_COUNT = 4000
TRAIN_RECT_SIZE = RECT_COUNT * 80 // 100
VAL_RECT_SIZE = RECT_COUNT - TRAIN_RECT_SIZE

CLASS_COUNT = 6
CLASSES_SQUARE = [0, 4, 5]
CLASSES_RECT = list(set(range(CLASS_COUNT)) - set(CLASSES_SQUARE))

print(CLASSES_SQUARE)
print(CLASSES_RECT)

[0, 4, 5]
[1, 2, 3]


In [4]:
def read_label_map(path):
    with open(path, 'r') as fin:
        reader = csv.reader(fin, delimiter=',')
        # ignore header
        next(reader)
        label_map = {filename: int(label) for filename, label in reader}
    return label_map


def get_images_paths(root, labels):
    dir_paths = list(map(lambda label: join(root, str(label)), labels))
    file_paths = [list(map(lambda filename: join(dir_path, filename), os.listdir(dir_path))) for dir_path in dir_paths]
    return roundrobin(*file_paths)


def get_train_data(path_images, labels, label_map):
    paths = repeat_infinitely(get_images_paths, path_images, labels)
    for path in paths:
        image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f'Image cannot be read: {path}')
        label = labels.index(label_map[os.path.basename(path)])
        yield (np.expand_dims(image, 0),
               to_categorical(label, len(labels))[0])


def collect_batches(iterable, batch_size=32, randomize=False, probability=0.5, rotate=False):
    while True:
        images, labels = [], []
        while len(images) < batch_size:
            image, label = next(iterable)
            if not randomize or np.random.rand() < probability:
                if rotate:
                    image = np.rot90(image, np.random.randint(0, 4), (1, 2))
                images.append(image)
                labels.append(label)
        yield np.array(images), np.array(labels)

In [5]:
label_map = read_label_map(join(PATH_DATA, 'train_labels.csv'))
gen_square_train_data = get_train_data(join(PATH_DATA, 'train'), CLASSES_SQUARE, label_map)
gen_rect_train_data = get_train_data(join(PATH_DATA, 'train'), CLASSES_RECT, label_map)
gen_square_val_data = get_train_data(join(PATH_DATA, 'validation'), CLASSES_SQUARE, label_map)
gen_rect_val_data = get_train_data(join(PATH_DATA, 'validation'), CLASSES_RECT, label_map)

gen_square_train_batches = collect_batches(gen_square_train_data, BATCH_SIZE, randomize=True, rotate=True)
gen_rect_train_batches = collect_batches(gen_rect_train_data, BATCH_SIZE, randomize=True, rotate=True)
gen_square_val_batches = collect_batches(gen_square_val_data, BATCH_SIZE, randomize=False, rotate=False)
gen_rect_val_batches = collect_batches(gen_rect_val_data, BATCH_SIZE, randomize=False, rotate=False)

In [6]:
# x, y = next(gen_rect_train_batches)
# print(x.shape)
# print(y)

# from matplotlib import pyplot as plt
# plt.imshow(x[0, 0], cmap='Greys_r')
# plt.show()

In [7]:
model_square = build_model(len(CLASSES_SQUARE), is_train=True)
model_rect = build_model(len(CLASSES_RECT), is_train=True)

model_square.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
model_rect.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

path_weights = join(PATH_PROJECT, 'backups')

In [8]:
model_square.fit_generator(generator=gen_square_train_batches,
                           steps_per_epoch=TRAIN_SQUARE_SIZE,
                           epochs=20,
                           validation_data=gen_square_val_batches,
                           validation_steps=VAL_SQUARE_SIZE,
                           callbacks=[ReduceLROnPlateau(patience=3),
                                      ModelCheckpoint(join(path_weights, 'weights_square_{epoch:02d}'))])

Epoch 1/20
2400/2400 [==============================] - 37s - loss: 0.0304 - acc: 0.9892 - val_loss: 2.2560e-07 - val_acc: 1.0000
Epoch 2/20
2400/2400 [==============================] - 33s - loss: 6.3925e-06 - acc: 1.0000 - val_loss: 1.2139e-07 - val_acc: 1.0000
Epoch 3/20
2400/2400 [==============================] - 34s - loss: 0.0070 - acc: 0.9988 - val_loss: 1.5527e-07 - val_acc: 1.0000
Epoch 4/20
2400/2400 [==============================] - 33s - loss: 1.2810e-07 - acc: 1.0000 - val_loss: 1.3918e-07 - val_acc: 1.0000
Epoch 5/20
2400/2400 [==============================] - 33s - loss: 1.5892e-07 - acc: 1.0000 - val_loss: 1.2964e-07 - val_acc: 1.0000
Epoch 6/20
2400/2400 [==============================] - 33s - loss: 1.2914e-07 - acc: 1.0000 - val_loss: 1.2805e-07 - val_acc: 1.0000
Epoch 7/20
2400/2400 [==============================] - 33s - loss: 1.3034e-07 - acc: 1.0000 - val_loss: 1.2676e-07 - val_acc: 1.0000
Epoch 8/20
2400/2400 [==============================] - 33s - loss: 1.

In [9]:
model_rect.fit_generator(generator=gen_rect_train_batches,
                         steps_per_epoch=TRAIN_RECT_SIZE,
                         epochs=20,
                         validation_data=gen_rect_val_batches,
                         validation_steps=VAL_RECT_SIZE,
                         callbacks=[ReduceLROnPlateau(patience=3),
                                    ModelCheckpoint(join(path_weights, 'weights_rect_{epoch:02d}'))])

Epoch 1/20
3200/3200 [==============================] - 154s - loss: 0.0510 - acc: 0.9772 - val_loss: 3.0726e-07 - val_acc: 1.0000
Epoch 2/20
3200/3200 [==============================] - 126s - loss: 1.4761e-07 - acc: 1.0000 - val_loss: 1.6719e-07 - val_acc: 1.0000
Epoch 3/20
3200/3200 [==============================] - 127s - loss: 1.2716e-07 - acc: 1.0000 - val_loss: 1.5065e-07 - val_acc: 1.0000
Epoch 4/20
3200/3200 [==============================] - 128s - loss: 1.3109e-07 - acc: 1.0000 - val_loss: 1.6041e-07 - val_acc: 1.0000
Epoch 5/20
3200/3200 [==============================] - 130s - loss: 1.2478e-07 - acc: 1.0000 - val_loss: 1.2837e-07 - val_acc: 1.0000
Epoch 6/20
3200/3200 [==============================] - 128s - loss: 1.2120e-07 - acc: 1.0000 - val_loss: 1.2934e-07 - val_acc: 1.0000
Epoch 7/20
3200/3200 [==============================] - 125s - loss: 1.2180e-07 - acc: 1.0000 - val_loss: 1.3031e-07 - val_acc: 1.0000
Epoch 8/20
3200/3200 [==============================] - 127

In [10]:
model_square_predict = build_model(len(CLASSES_SQUARE), is_train=False)
model_rect_predict = build_model(len(CLASSES_RECT), is_train=False)

model_square_predict.load_weights(join(path_weights, 'weights_square_19'))
model_rect_predict.load_weights(join(path_weights, 'weights_rect_19'))

In [11]:
path_test = join(PATH_DATA, 'test')
answers = {}
for filename in os.listdir(path_test):
    file_path = join(path_test, filename)
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    model_predict = model_square_predict
    classes = CLASSES_SQUARE
    if image.shape[0] != image.shape[1]:
        model_predict = model_rect_predict
        classes = CLASSES_RECT
    tensor = image.reshape((1, 1, *image.shape))
    label_categorical = model_predict.predict(tensor)
    label = np.argmax(label_categorical)
    answers[filename] = classes[label]

In [12]:
with open(join(PATH_PROJECT, 'submission.csv'), 'w') as fout:
    for filename, label in answers.items():
        print(filename, label, file=fout, sep=',')